## Setup


In [ ]:
## Package imports and environment sanity check
import sys
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import numpy as np
import json
from scipy.stats import chi2
import warnings
warnings.filterwarnings(
    "ignore",
    message="The bic value is computed using the deviance formula"
)
from termcolor import colored

In [ ]:
# Display settings for pandas
pd.set_option('display.width', 200)          # total characters per line before wrapping
pd.set_option('display.max_columns', None)   # show all columns
pd.set_option('display.max_colwidth', None)  # don’t truncate long text

In [ ]:
## Load Data and read data
df_hist = pd.read_excel('data/Interview_data_2020_2024.xlsx')
df_2025 = pd.read_excel('data/Interview_data_2025.xlsx')

df_hist['frequency'] = df_hist['claims_number'] / df_hist['exposure']
df_hist['dwelling_type'] = df_hist['dwelling_type'].astype('str')
df_hist['dwelling_type'] = df_hist['dwelling_type'].astype('str')

df_2025['dwelling_type'] = df_hist['building_class'].astype('str')
df_2025['dwelling_type'] = df_hist['dwelling_type'].astype('str')

df_all = pd.concat([df_hist, df_2025], ignore_index=True)

## Exploratory Data Analysis

### Structural checks and data quality

In [ ]:
print('Datafframe sample:')
display(df_all.head(10))

In [ ]:
# Check for  shape, column dtypes, memory usage and non-null counts
print('Dataframe formats information:\n')
display(df_all.info())


In [ ]:
print("Dataframe number of uniques:")
df_all.nunique()

In [ ]:
print('Number of duplicated rows:', df_all.duplicated().sum())
print('Number of negative exposures in:', (df_all['exposure'] < 0).sum())

if df_all.duplicated().sum() > 0:
    print('Duplicated rows in the dataframe:')
elif (df_all['exposure']<0).sum() > 0:
    print('Rows with negative exposure values:\n', df_hist[df_hist['exposure'] < 0])


### Distribution checks

In [ ]:
print('Dataframe features metrics description:')
df_all.describe()

In [ ]:
print('Claims number, Exposure, and Frequency distributions:\n')
fig, axes = plt.subplots(1, 3, figsize=(17, 4))
plt.figure(figsize=(5,3))
sns.histplot(df_hist['claims_number'], bins=30, kde=False, ax=axes[0]).set_title('Claims Number Distribution')
sns.histplot(df_hist['exposure'], bins=30, kde=False, ax=axes[1]).set_title('Exposure Distribution')
sns.histplot(df_hist['frequency'], bins=30, kde=False, ax=axes[2]).set_title('Frequency Distribution')
plt.tight_layout()
plt.show() 

### Relationship checks

In [ ]:
corr = df_hist.corr(numeric_only=True)
plt.figure(figsize=(8,5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, cbar=True, annot_kws={"size": 7})
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

#### Observations
- Almost no correlation between factors which offers flexibility for feature selections
- Weak but positive correlation between 
    - area and claims_number (0.08);   
    - exposure and year (0.05)

In [ ]:
# Sum exposure by each level
exp_building = df_hist.groupby("building_class")["exposure"].sum().reset_index()
exp_building["Variable"] = "Building Class"
exp_building.rename(columns={"building_class": "Level"}, inplace=True)

exp_dwelling = df_hist.groupby("dwelling_type")["exposure"].sum().reset_index()
exp_dwelling["Variable"] = "Dwelling Type"
exp_dwelling.rename(columns={"dwelling_type": "Level"}, inplace=True)

bins = [0, 500, 1000, 2000, 30000]
labels = [f"[{bins[i]},{bins[i+1]})" for i in range(len(bins)-1)]
df_hist["area_band"] = pd.cut(df_hist["area"], bins=bins, right=False, labels=labels, include_lowest=True)

exp_area = df_hist.groupby("area_band")["exposure"].sum().reset_index()
exp_area["Variable"] = "Area Band"
exp_area.rename(columns={"area_band": "Level"}, inplace=True)

# Combine all into one dataframe
df_exp_all = pd.concat([exp_building, exp_dwelling, exp_area], ignore_index=True)

# Plot all together
plt.figure(figsize=(10, 5))
sns.barplot(data=df_exp_all, x="Level", y="exposure", hue="Variable", estimator=sum)
plt.title("Exposure by level across all variables", fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Observations
We can already expect base levels per risk factor (if selected) to be:  
- Building Class: 1  
- Dwelling type: HR 
- Area band: [1000, 2000). Area bands are defined by judgement and interpretability.

In [ ]:
print("Claims number, exposure and frequency per year:")

g = (
    df_hist.groupby('year', as_index=True)
           .agg(claims_number=('claims_number','sum'),
                exposures=('exposure','sum'))
           .sort_index()
)

g['frequency'] = g['claims_number'] / g['exposures'].replace(0, np.nan)

for col in ['claims_number', 'exposures', 'frequency']:
    g[f'{col}_yoy'] = (g[col].pct_change() * 100).round(1)

print(
    g.round({
        'claims_number': 0,
        'exposures': 1,
        'frequency': 6,          # keep precision; adjust as needed
        'claims_number_yoy': 1,
        'exposures_yoy': 1,
        'frequency_yoy': 1
    })
)

fig, axes = plt.subplots(1, 3, figsize=(17, 4), constrained_layout=True)

sns.lineplot(data=g.reset_index(), x="year", y="claims_number", marker="o", ax=axes[0])
axes[0].set_title('Claims by year')

sns.lineplot(data=g.reset_index(), x="year", y="exposures", marker="o", ax=axes[1])
axes[1].set_title('Exposure by year')

sns.lineplot(data=g.reset_index(), x="year", y="frequency", marker="o", ax=axes[2])
axes[2].set_title('Frequency by year')

for ax in axes:
    ax.set_xticks(sorted(g.reset_index()['year'].unique()))  # one tick per year
    ax.grid(True, alpha=0.3)

plt.show()

#### Observations
- Between 2020 and 2024, exposure increases steadily yoy while claim counts drops in 2021 (+12.5%) and peaks again in 2022 (+28.8%) vs ~17% average growth.
- Similarly to claims_number, increasing frequency between 2020 and 2024 but smoother pattern. Recent increase of +3.1% (as per average).

In [ ]:
print("Frequency by risk factor 'building_class'")

freq_by_class = (
    df_hist.groupby(['building_class','year'], as_index=True)
           .agg(claims=('claims_number','sum'),
                exp=('exposure','sum'))
           .sort_index()
)

freq_by_class['freq'] = freq_by_class['claims'] / freq_by_class['exp'].replace(0, np.nan)

for col in ['claims', 'exp', 'freq']:
    freq_by_class[f'{col}_yoy'] = (freq_by_class.groupby(level=0)[col].pct_change() * 100).round(1)

print('freq_by_class with YoY changes:\n', freq_by_class.round(3))

fig, axes = plt.subplots(1, 3, figsize=(17, 4), constrained_layout=True)
sns.lineplot(data=freq_by_class.reset_index(), x="year", y="claims", hue="building_class", marker="o", ax=axes[0])
axes[0].set_title('Claims by building_class and year')

sns.lineplot(data=freq_by_class.reset_index(), x="year", y="exp", hue="building_class", marker="o", ax=axes[1])
axes[1].set_title('Exposure by building_class and year')

sns.lineplot(data=freq_by_class.reset_index(), x="year", y="freq", hue="building_class", marker="o", ax=axes[2])
axes[2].set_title('Frequency by building_class and year')

years = sorted(df_hist['year'].unique())
for ax in axes:
    ax.set_xticks(years)
    ax.grid(True, alpha=0.3)
    leg = ax.get_legend()
    if leg is not None:
        leg.set_title('Building class')
        leg.set_frame_on(False)

plt.show()

#### Observations
Analyzing growth by building class, exposure again increases steadily between 2020 and 2024. Claim counts again spikes in 2022 with +29.4% (Class 1), +32.4% (Class 2), and +24.0% (Class 3). Due to a slower incresaing exposure, frequency is less volatile than claims counts, but similarly increasing:
- Class 1 frequency (highest exposure) is second most stable: Steady, moderate increases but converging (3.6% most recently vs. 3.7% average)
- Class 2 frequency shows the most volatility: Massive spikes in claims (32.4% in 2022, 25.1% in 2024) with corresponding frequency jumps (17.3% in 2022, 13.1% in 2024 vs. ~+7% average)
- Class 3 frequency is most stable and recently improved: 2024 saw -7.3% yoy (vs. ~+1.2% average) despite continued exposure growth.

In [ ]:
print("Frequency by risk factor 'dwelling_type'")

freq_by_dwelling_type = (
    df_hist.groupby(['dwelling_type','year'], as_index=True)
           .agg(claims=('claims_number','sum'),
                exp=('exposure','sum'))
           .sort_index()
)

freq_by_dwelling_type['freq'] = freq_by_dwelling_type['claims'] / freq_by_dwelling_type['exp'].replace(0, np.nan)

for col in ['claims', 'exp', 'freq']:
    freq_by_dwelling_type[f'{col}_yoy'] = (freq_by_dwelling_type.groupby(level=0)[col].pct_change() * 100).round(1)

print('freq_by_dwelling_type with YoY changes:\n', freq_by_dwelling_type.round(3))

fig, axes = plt.subplots(1, 3, figsize=(17, 4), constrained_layout=True)
sns.lineplot(data=freq_by_dwelling_type.reset_index(), x="year", y="claims", hue="dwelling_type", marker="o", ax=axes[0])
axes[0].set_title('Claims by dwelling_type and year')

sns.lineplot(data=freq_by_dwelling_type.reset_index(), x="year", y="exp", hue="dwelling_type", marker="o", ax=axes[1])
axes[1].set_title('Exposure by dwelling_type and year')

sns.lineplot(data=freq_by_dwelling_type.reset_index(), x="year", y="freq", hue="dwelling_type", marker="o", ax=axes[2])
axes[2].set_title('Frequency by dwelling_type and year')

years = sorted(df_hist['year'].unique())
for ax in axes:
    ax.set_xticks(years)
    ax.grid(True, alpha=0.3)
    leg = ax.get_legend()
    if leg is not None:
        leg.set_title('Dwelling type')
        leg.set_frame_on(False)

plt.show()

#### Observations
- Whilst exposure grows steadily btw. 2020 and 2024, claims number grow similarly except for the 2022 spike of +29.0% (dt=BR) and +28.5% (dt=HR).
- BR: Consistently higher baseline frequency (~0.035-0.041), experienced sharp 2021 drop (-8.1%) and 2022 spike (+14.4%).
- HR (Highest exposure): Lower baseline frequency (~0.022-0.026), experienced sharp 2022 spike (+13.8%).
- Recent trend: Both types converging - BR accelerating (+4.3% freq in 2024 vs 3.8% average) while HR stabilizing (+2.0% vs 4.1% average)

In [ ]:
print("Frequency by risk factor 'area_band'")

freq_by_area_band = (
    df_hist.groupby(['area_band','year'], as_index=True)
           .agg(claims=('claims_number','sum'),
                exp=('exposure','sum'))
           .sort_index()
)

freq_by_area_band['freq'] = freq_by_area_band['claims'] / freq_by_area_band['exp'].replace(0, np.nan)

for col in ['claims', 'exp', 'freq']:
    freq_by_area_band[f'{col}_yoy'] = (freq_by_area_band.groupby(level=0)[col].pct_change() * 100).round(1)

print('freq_by_area_band with YoY changes:\n', freq_by_area_band.round(3))

fig, axes = plt.subplots(1, 3, figsize=(17, 4), constrained_layout=True)
sns.lineplot(data=freq_by_area_band.reset_index(), x="year", y="claims", hue="area_band", marker="o", ax=axes[0])
axes[0].set_title('Claims by area_band and year')

sns.lineplot(data=freq_by_area_band.reset_index(), x="year", y="exp", hue="area_band", marker="o", ax=axes[1])
axes[1].set_title('Exposure by area_band and year')

sns.lineplot(data=freq_by_area_band.reset_index(), x="year", y="freq", hue="area_band", marker="o", ax=axes[2])
axes[2].set_title('Frequency by area_band and year')

years = sorted(df_hist['year'].unique())
for ax in axes:
    ax.set_xticks(years)
    ax.grid(True, alpha=0.3)
    leg = ax.get_legend()
    if leg is not None:
        leg.set_title('Area band')
        leg.set_frame_on(False)

plt.show()

#### Observations
- Greater area_bands lead for higher frequencies and thus highest frequency in area band [2000, 30000).
- Larger properties ([2000,30000)): 
    - Highest baseline frequency (0.054-0.060)
    - Relatively stable growth pattern and improving in 2024 (-3.3% freq in 2024 vs +2.2% average)
- ([1000,2000)) (Highest exposure):
    - Moderate frequency (0.025-0.034)
    - Accelerating upward trend (+6.8% freq in 2024 vs 4.2% average)
- Small properties ([0,500) and [500,1000)):
    - Lowest frequency historically
    - BUT showing extreme volatility in recent years
        - [0,500) band actually declined in 2024 (-13.1% frequency vs 3.3% average)
        - [500,1000) spiked massively (+27.2% claims, +15.6% frequency in 2024 vs 8.75% average)

In [ ]:
# Handling outliers
outliers = df_hist[df_hist['frequency'] > 2]
print('Number of policies with freq > 2:', len(outliers))
print(outliers[['policy_id','year','building_class','dwelling_type','exposure','claims_number','frequency']].head(20))

### Insight 1
Area Band Shows the Clearest Pattern  
1. Widest Range of Risk Profiles  
    The area bands show a 6-7x difference in baseline frequency:  
    - Small properties ([0,500)): ~0.009-0.011 frequency  
    - Large properties ([2000,30000)): ~0.054-0.060 frequency  

    Compared to:  
    - Building class: Only ~1.5x difference (0.025 to 0.037)  
    - Dwelling type: Only ~1.6x difference (0.022 to 0.041)  

    -> This wider spread makes it easier to see distinct behavioral patterns across segments  

2. More Distinct Behavioral Patterns  
    Each area band tells a different story:  
    [0,500): only 55 claims in 2024
    [500,1000): Explosive recent growth - emerging situation
    [1000,2000): Steady, predictable increases - the "stable increasing middle"  
    [2000,30000): High but stable frequency - well-understood risk  

    -> Versus building class/dwelling type where patterns are more similar across segments and baseline levels are converging.

3. Clearer Divergence in Recent Years  
    2024 frequency changes:  
    [0,500): -13.1%  
    [500,1000): +15.6%
    [1000,2000): +6.8%  
    [2000,30000): -3.3%  

    -> This shows four completely different trajectories, suggesting area/value is a stronger predictor of claims behavior than the other variables 

4. Trend  
    Given the overall upward trend, the year variable should be considered as an aditional explanatory variable

### Temporal stability

In [ ]:
## Portofolio Mix Shift Analysis
# Count policies by year, class, dwelling
counts = (
    df_hist.groupby(['year','building_class', 'dwelling_type', 'area_band'])
           .size()
           .rename('policies')
)

# Normalize within each year → share of total
mix_shift = counts / counts.groupby(level='year').transform('sum')
mix_shift = mix_shift.reset_index()

mix_pivot = mix_shift.pivot_table(
    index='year', 
    columns=['building_class','dwelling_type', 'area_band'],
    values='policies'
)

mix_pivot.plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab20')
plt.title('Portfolio Mix Shift by Year (Building Class × Dwelling Type × Area Band)', fontsize=14)
plt.ylabel('Share of Portfolio')
plt.xlabel('Year')
plt.legend(title='Class × Type', bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.figure(figsize=(4,3))
plt.show()

## Model Development

In [ ]:
# Helpers 
def add_area_band(df):
    return pd.cut(df['area'], bins=bins, right=False)

def _one_hot(df, cols):
    df_ = df.copy()
    for c in cols:
        df_[c] = df_[c].astype('category')
    return pd.get_dummies(df_[cols], drop_first=True, dtype=float)

def _interaction_cols(A: pd.DataFrame, B: pd.DataFrame, prefix='I_'):
    out = {}
    for a in A.columns:
        for b in B.columns:
            out[f'{prefix}{a}:@:{b}'] = A[a].values * B[b].values
    return pd.DataFrame(out, index=A.index) if out else pd.DataFrame(index=A.index)

def _three_way_cols(A: pd.DataFrame, B: pd.DataFrame, C: pd.DataFrame, prefix='I_'):
    out = {}
    for a in A.columns:
        for b in B.columns:
            for c in C.columns:
                out[f'{prefix}{a}:@:{b}:@:{c}'] = A[a].values * B[b].values * C[c].values
    return pd.DataFrame(out, index=A.index) if out else pd.DataFrame(index=A.index)

def make_X(df, use_area_band: bool, include_bc: bool, include_dw: bool, interactions: set):
    '''
    interactions ∈ {'bcXdw','bcXarea','dwXarea','bcXdwXarea'}
    Only applied if parent main effects are present.
    '''
    df_c = df.copy()

    # Main effects (optional)
    Z_parts = []

    bc_block = pd.DataFrame(index=df_c.index)
    dw_block = pd.DataFrame(index=df_c.index)

    if include_bc:
        bc_block = _one_hot(df_c, ['building_class'])[
            [c for c in _one_hot(df_c, ['building_class']).columns if c.startswith('building_class_')]
        ]
        Z_parts.append(bc_block)

    if include_dw:
        dw_block = _one_hot(df_c, ['dwelling_type'])[
            [c for c in _one_hot(df_c, ['dwelling_type']).columns if c.startswith('dwelling_type_')]
        ]
        Z_parts.append(dw_block)

    # Area (always include, linear OR banded)
    if use_area_band:
        area_block = pd.get_dummies(add_area_band(df_c), drop_first=True, prefix='area', dtype=float)
        area_is_linear = False
    else:
        area_block = pd.DataFrame({'area': df_c['area'].astype(float)})
        area_is_linear = True
    Z_parts.append(area_block)

    Z = pd.concat(Z_parts, axis=1) if Z_parts else area_block.copy()

    # --- Interactions (guarded by parent presence) ---
    # bc × dw
    if 'bcXdw' in interactions and include_bc and include_dw:
        Z = Z.join(_interaction_cols(bc_block, dw_block, prefix='I_'))

    # bc × area
    if 'bcXarea' in interactions and include_bc:
        if area_is_linear:
            inter = {f'I_{b}:@:area': bc_block[b].values * area_block['area'].values for b in bc_block.columns}
            if inter: Z = Z.join(pd.DataFrame(inter, index=Z.index))
        else:
            Z = Z.join(_interaction_cols(bc_block, area_block, prefix='I_'))

    # dw × area
    if 'dwXarea' in interactions and include_dw:
        if area_is_linear:
            inter = {f'I_{d}:@:area': dw_block[d].values * area_block['area'].values for d in dw_block.columns}
            if inter: Z = Z.join(pd.DataFrame(inter, index=Z.index))
        else:
            Z = Z.join(_interaction_cols(dw_block, area_block, prefix='I_'))

    # three-way bc × dw × area
    if 'bcXdwXarea' in interactions and include_bc and include_dw:
        if area_is_linear:
            pair = _interaction_cols(bc_block, dw_block, prefix='I_')
            for col in pair.columns:
                pair[col+':@:area'] = pair[col].values * area_block['area'].values
            keep = [c for c in pair.columns if c.endswith(':@:area')]
            if keep: Z = Z.join(pair[keep])
        else:
            Z = Z.join(_three_way_cols(bc_block, dw_block, area_block, prefix='I_'))

    # Constant & dtype
    Z = sm.add_constant(Z, has_constant='add')
    return Z.astype(float).reset_index(drop=True)

# --- Time trend helper (numeric time instead of year dummies) ---
def add_time_trend(X: pd.DataFrame, df: pd.DataFrame, *,
                   base_year: int = None,
                   include_t: bool = True,
                   include_areaXt: bool = True,
                   area_banded: bool = True):
    """
    Adds a numeric time slope t = year - base_year to the design matrix X.
    Optionally adds area × t interaction (banded or linear).

    Parameters
    ----------
    X : design matrix from make_X (already has const, mains, and non-time interactions)
    df : same rows as X, must contain 'year' and (if area_banded=False) 'area'
    base_year : anchor; if None uses df['year'].min()
    include_t : include plain time slope (t)
    include_areaXt : include area × time slope
    area_banded : True if area is represented by dummies in X, False if linear 'area' column
    """
    if 'year' not in df.columns:
        raise ValueError("add_time_trend: df must have a 'year' column.")

    by = int(df['year'].min()) if base_year is None else int(base_year)
    t = (df['year'] - by).astype(float)

    X = X.copy()
    if include_t and ('t' not in X.columns):
        X['t'] = t

    if include_areaXt:
        if area_banded:
            area_cols = [c for c in X.columns if c.startswith('area_[')]
            for c in area_cols:
                X[f"I_{c}:@:t"] = X[c] * t
        else:
            if 'area' not in X.columns:
                raise ValueError("add_time_trend: area_banded=False requires 'area' column in X.")
            X['I_area:@:t'] = X['area'] * t

    return X, by

#  Fit chooser & diagnostics
def fit_glm_poisson_or_nb(y, X, exposure):
    """Fit Poisson; if overdispersed (>1.3), try NB and return chosen family, result, and Poisson dispersion."""
    # align and enforce numeric
    y = y.astype(int).reset_index(drop=True)
    X = X.astype(float).reset_index(drop=True)
    exposure = exposure.reset_index(drop=True)

    # guard against zero/neg exposures
    if (exposure <= 0).any():
        raise ValueError("Exposure must be strictly positive for log-offset.")

    # Poisson fit
    mP = sm.GLM(y, X, family=sm.families.Poisson(), offset=np.log(exposure))
    resP = mP.fit()
    disp = float(resP.deviance / resP.df_resid)  # Poisson dispersion check

    # If materially overdispersed, try NB
    if disp > 1.3:
        mNB = sm.GLM(y, X, family=sm.families.NegativeBinomial(), offset=np.log(exposure))
        resNB = mNB.fit()
        return ("NB", resNB, disp)

    return ("Poisson", resP, disp)

def normalized_gini(actual, pred):
    """Normalized (relative) Gini — robust & simple."""
    # Sort by prediction desc
    order = np.argsort(-pred)
    a = np.asarray(actual)[order]
    # Lorenz area for model
    cum_a = a.cumsum()
    lor_model = cum_a / cum_a[-1] if cum_a[-1] != 0 else cum_a
    x = np.arange(1, len(a)+1) / len(a)
    g_model = np.trapezoid(lor_model, x) - 0.5
    # Lorenz area for random (pred=constant) equals 0.5 baseline, so normalize vs perfect
    # Compute "perfect" ordering by actual
    order_star = np.argsort(-np.asarray(actual))
    a_star = np.asarray(actual)[order_star]
    cum_star = a_star.cumsum()
    lor_star = cum_star / cum_star[-1] if cum_star[-1] != 0 else cum_star
    g_star = np.trapezoid(lor_star, x) - 0.5
    return (g_model / g_star) if g_star != 0 else 0.0

def collect_metrics(res, y, mu, X):
    """Return a dict with AIC, BIC, dispersion, in-sample Poisson deviance, normalized Gini."""
    n = len(y)
    k = len(res.params)  # includes intercept
    # AIC direct from statsmodels; BIC: use res.bic if present else compute
    aic = float(res.aic)
    try:
        bic = float(res.bic)
    except Exception:
        bic = (np.log(n) * k) - 2.0 * float(res.llf)

    # Pearson chi-square / df (dispersion proxy)
    # statsmodels provides res.pearson_chi2
    pearson_over_df = float(res.pearson_chi2 / res.df_resid)

    # Gini on expected counts (ranking power)
    gini = normalized_gini(y, mu)

    return {
        "AIC": round(aic, 1),
        "BIC": round(bic, 1),
        "dispersion_Poisson": round(pearson_over_df, 3),
        "gini_normalized": round(gini, 3),
        "k_parameters": k
    }

def fmt(x, width=9, prec=1):
    try:
        return f"{float(x):>{width}.{prec}f}"
    except Exception:
        return f"{str(x):>{width}}"

## Model Selection

In [ ]:
y   = df_hist['claims_number']
exp = df_hist['exposure']

# --- Main and interaction design grid ---
main_effects_grid = [
    (False, False),  # area only
    (True,  False),  # bc only
    (False, True),   # dw only
    (True,  True),   # bc + dw
]

interaction_menu = [
    set(), {'bcXdw'}, {'bcXarea'}, {'dwXarea'},
    {'bcXdw','bcXarea'}, {'bcXdw','dwXarea'}, {'bcXarea','dwXarea'},
    {'bcXdw','bcXarea','dwXarea'},
    {'bcXdwXarea'}, {'bcXdw','bcXdwXarea'}, {'bcXarea','bcXdwXarea'},
    {'dwXarea','bcXdwXarea'}, {'bcXdw','bcXarea','dwXarea','bcXdwXarea'},
]

MAX_PARAMS = 200
candidates, model_num = [], 0

print(colored("=== MODEL CANDIDATES (subsets + interactions) ===", "cyan"))
print(f"{'No':<4} {'Spec':<60} {'Fam':<8} {'AIC':>9} {'BIC':>11} {'Pearson/df':>10} {'Gini':>6} {'k':>4}")

for use_area_band in [False, True]:
    for include_bc, include_dw in main_effects_grid:
        for inter_set in interaction_menu:

            # Skip impossible parentless interactions
            if ('bcXdw' in inter_set or 'bcXdwXarea' in inter_set) and not (include_bc and include_dw):
                continue
            if ('bcXarea' in inter_set) and not include_bc:
                continue
            if ('dwXarea' in inter_set) and not include_dw:
                continue

            X = make_X(df_hist, use_area_band, include_bc, include_dw, inter_set)
            k = X.shape[1]
            model_num += 1

            spec_txt = (
                f"area={'banded' if use_area_band else 'linear'} | "
                f"bc={'Y' if include_bc else 'N'} | "
                f"dw={'Y' if include_dw else 'N'} | "
                f"inter={'+'.join(sorted(inter_set)) or 'none'}"
            )

            if k > MAX_PARAMS:
                print(f"{model_num:<4} {spec_txt[:58]:<60} {'SKIP':<8} {'—':>9} {'—':>11} {'—':>10} {'—':>6} {k:>4}")
                continue

            fam, res, _ = fit_glm_poisson_or_nb(y, X, exp)
            mu = res.predict(X, offset=np.log(exp))
            m  = collect_metrics(res, y.values, mu, X)

            candidates.append({
                'spec': spec_txt,
                'family': fam,
                **m,
                'k': k,
                'res': res
            })

            print(f"{model_num:<4} {(spec_txt[:58] + '…') if len(spec_txt) > 58 else spec_txt:<60} "
                  f"{fam:<8} {fmt(m['AIC']):>9} {fmt(m['BIC']):>11} "
                  f"{fmt(m['dispersion_Poisson'], prec=3):>10} "
                  f"{fmt(m['gini_normalized'], prec=3):>6} {m['k_parameters']:>4}")

# --- Ranking and summary ---
def rank_key(c):
    area_is_banded = 'area=banded' in c['spec']
    bcY = 'bc=Y' in c['spec']
    dwY = 'dw=Y' in c['spec']
    inter_token = c['spec'].split('inter=')[-1]
    inter_count = 0 if inter_token == 'none' else inter_token.count('+') + 1
    mains_count = int(bcY) + int(dwY)
    return (c['AIC'], area_is_banded, mains_count, inter_count, c['k'])

ranked = sorted(candidates, key=rank_key)
choice = ranked[1]
choice_res = choice['res']

# --- Top 10 Table ---
top15 = pd.DataFrame([
    {
        'spec': c['spec'],
        'family': c['family'],
        'AIC': round(c['AIC'], 1),
        'BIC': round(c['BIC'], 1),
        'Pearson/df': round(c['dispersion_Poisson'], 3),
        'Gini': round(c['gini_normalized'], 3),
        'k': c['k']
    }
    for c in ranked[:15]
])

print("\n" + colored("=== TOP 10 BEST MODELS (sorted by AIC) ===", "magenta"))
print(top15.to_string(index=False))

# --- Chosen candidate summary ---
print("\n" + colored("=== CHOSEN CANDIDATE MODEL ===", "green"))
print(f"{choice['spec']} | Family: {choice['family']} | "
      f"AIC={choice['AIC']:.1f} | BIC={choice['BIC']:.1f} | "
      f"Pearson/df={choice['dispersion_Poisson']:.3f} | "
      f"Gini={choice['gini_normalized']:.3f} | k={choice['k']}")

print("\n" + colored("=== FINAL MODEL SUMMARY ===", "yellow"))
print(choice_res.summary())

#### Observations
a. Clear superiority of the banded area specification.
Top 10 best-performing models (AIC ≈ 41 550–41 580) use area=banded.
- This confirms that non-linearity in area (size segmentation) captures risk much better than treating area as a continuous variable.
- Note that bins choice directly impacts this observation. Previously used bins should be considered.

b. Importance of both building_class and dwelling_type.
Top 10 candidates include both (bc=Y | dw=Y).
- Dropping either leads to a ~120 AIC deterioration (vs. best candidate).
- These categorical effects are essential.

c. Small incremental gains from second-order interactions.
Best model: inter=bcXdw+dwXarea.
Next best: bcXdw alone (ΔAIC ≈ 0.4).
- Adding both interactions only slightly improves AIC/BIC.
- Third-order (bcXdwXarea) brings no gain and risks overfitting (>k).

d. Good dispersion control.
Pearson/df ≈ 1.00 across top models, means no strong over/under-dispersion. 
- Poisson family remains statistically valid and Negative Binomial not needed.

e. Gini stability.
Gini ≈ 0.33–0.34 is consistent across top models, confirming predictive ordering doesn’t change much. 
- Choice among top few specs won’t alter ranking behaviour.

f. Coefficients significance.
- 8 out of 9 terms are statistically significant.
- The only non-significant effect is building_class_2, meaning only there’s no strong evidence that class 2 differs from the reference (class 1).

## Model Validation

In [ ]:
# Spec to validate
USE_AREA_BANDED = True          # area=banded
INCLUDE_BC = True               # bc=Y
INCLUDE_DW = True               # dw=Y
BASE_INTERACTIONS = {'bcXdw'}   # include bcXdw

# Base design matrix
y   = df_hist['claims_number']
exp = df_hist['exposure']

X_base = make_X(
    df_hist,
    use_area_band=USE_AREA_BANDED,
    include_bc=INCLUDE_BC,
    include_dw=INCLUDE_DW,
    interactions=BASE_INTERACTIONS
)
if 'const' in X_base.columns:
    X_base['const'] = 1.0

# Fit base model (no time interaction)
base_res = sm.GLM(y, X_base, family=sm.families.Poisson(), offset=np.log(exp)).fit()
base_llf = base_res.llf
base_aic = base_res.aic

# Utilities to add time (year) interactions safely
def _year_block(df):
    return pd.get_dummies(df['year'].astype('category'),
                          drop_first=True, prefix='year', dtype=float)

def _select_cols(Z, prefix):
    return [c for c in Z.columns if c.startswith(prefix)]

def add_time_interactions(X_base, df, which={'bcXyear','dwXyear','areaXyear'}):
    """Return X_time = X_base + requested year interactions (guards on parent presence)."""
    X_time = X_base.copy()
    Yblk = _year_block(df)

    bc_cols = _select_cols(X_base, 'building_class_')
    dw_cols = _select_cols(X_base, 'dwelling_type_')
    area_is_linear = ('area' in X_base.columns)
    area_cols = _select_cols(X_base, 'area_') if not area_is_linear else []

    # bc × year
    if 'bcXyear' in which and bc_cols:
        inter = {f"I_{b}:@:{y}": X_base[b].values * Yblk[y].values
                 for b in bc_cols for y in Yblk.columns}
        X_time = X_time.join(pd.DataFrame(inter, index=X_time.index))

    # dw × year
    if 'dwXyear' in which and dw_cols:
        inter = {f"I_{d}:@:{y}": X_base[d].values * Yblk[y].values
                 for d in dw_cols for y in Yblk.columns}
        X_time = X_time.join(pd.DataFrame(inter, index=X_time.index))

    # area × year
    if 'areaXyear' in which:
        if area_is_linear:
            inter = {f"I_area:@:{y}": X_base['area'].values * Yblk[y].values
                     for y in Yblk.columns}
        else:
            inter = {f"I_{a}:@:{y}": X_base[a].values * Yblk[y].values
                     for a in area_cols for y in Yblk.columns}
        X_time = X_time.join(pd.DataFrame(inter, index=X_time.index))

    if 'const' in X_time.columns:
        X_time['const'] = 1.0
    return X_time

# Fit time-interaction variants + LR Tests
tests = []

def fit_and_compare(X_new, label):
    res_new = sm.GLM(y, X_new, family=sm.families.Poisson(), offset=np.log(exp)).fit()
    LR      = 2 * (res_new.llf - base_llf)
    df_diff = res_new.df_model - base_res.df_model
    p       = chi2.sf(LR, df_diff)
    tests.append({
        'interaction_set': label,
        'k': int(res_new.df_model + 1),
        'AIC': float(res_new.aic),
        'ΔAIC': float(res_new.aic - base_aic),
        'LR_stat': float(LR),
        'df_diff': int(df_diff),
        'p_value': float(p)
    })
    return res_new

# Individually present interactions (guarded by parents)
which_individual = []
if INCLUDE_BC: which_individual.append({'bcXyear'})
if INCLUDE_DW: which_individual.append({'dwXyear'})
which_individual.append({'areaXyear'})  # always check area stability

fitted_variants = {}
for w in which_individual:
    X_tmp = add_time_interactions(X_base, df_hist, which=w)
    key = next(iter(w))
    fitted_variants[key] = fit_and_compare(X_tmp, key)

# ALL present
which_all = set()
if INCLUDE_BC: which_all.add('bcXyear')
if INCLUDE_DW: which_all.add('dwXyear')
which_all.add('areaXyear')
X_all = add_time_interactions(X_base, df_hist, which=which_all)
res_time = fit_and_compare(X_all, 'ALL')

time_val_df = pd.DataFrame(tests).sort_values('AIC')
print("\nTemporal interaction comparison vs BASE:")
print(time_val_df.to_string(index=False))

# Model-validation plots
df_pred = df_hist.copy()
df_pred['mu_hat'] = res_time.predict(X_all, offset=np.log(df_pred['exposure']))

def summarize_factor(df, factor):
    g = df.groupby([factor, 'year'], as_index=False).agg(
        mu_hat_sum=('mu_hat','sum'),
        exposure_sum=('exposure','sum')
    )
    g['freq_hat'] = g['mu_hat_sum'] / g['exposure_sum']  # rate
    return g

fig, axes = plt.subplots(1, 3, figsize=(17, 3), sharey=True)

for ax, factor in zip(axes, ["building_class", "dwelling_type", "area_band"]):
    df_fac = summarize_factor(df_pred, factor)

    # enforce ordered categories so seaborn doesn't reorder
    if factor == "building_class":
        order = ["1", "2", "3"]
        df_fac[factor] = df_fac[factor].astype(str)
    elif factor == "dwelling_type":
        order = ["BR", "HR"]
    else:
        if pd.api.types.is_categorical_dtype(df_fac[factor]):
            order = list(df_fac[factor].cat.categories)
        else:
            order = list(pd.unique(df_fac[factor]))
    df_fac[factor] = pd.Categorical(df_fac[factor], categories=order, ordered=True)

    #  left axis: log(expected frequency)
    sns.lineplot(
        data=df_fac,
        x=factor, y=np.log(df_fac['freq_hat']),
        hue='year', marker='o', ax=ax, palette='tab10',
        sort=False, estimator=None, legend=(ax is axes[0])
    )
    ax.set_xlabel(factor.replace('_',' ').title())
    ax.set_ylabel('log(Expected Frequency)')
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    ax.grid(True, linestyle='--', alpha=0.4)
    ax.set_title(factor.replace('_',' ').title())
    # clamp x so lines cannot extend beyond categories
    ax.set_xlim(-0.5, len(order) - 0.5)

    # right axis: exposure bars in same order
    ax2 = ax.twinx()
    sns.barplot(
        data=df_fac, x=factor, y='exposure_sum',
        order=order, ax=ax2, color='grey', alpha=0.2, errorbar=None
    )
    ax2.set_ylabel('Exposure (sum)')
    ax2.set_yscale('log')
    ax2.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x):,}"))

# shared legend (Year)
leg = axes[0].legend(title='Year', loc='upper left', framealpha=0.35, fontsize=8)
if leg is not None:
    leg.get_frame().set_edgecolor('none')

plt.tight_layout()
plt.show()

In [ ]:
## Residuals Validation
plt.figure(figsize=(6,4))
plt.hist(choice_res.resid_deviance, bins=40, edgecolor='black', alpha=0.7)
plt.title(f"Deviance residuals — {choice['spec']} ({choice['family']})")
plt.xlabel("Deviance residual")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

mu = choice_res.fittedvalues
resid = choice_res.resid_deviance

plt.figure(figsize=(6,4))
plt.scatter(mu, resid, alpha=0.4)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Fitted mean (μ̂)")
plt.ylabel("Deviance residuals")
plt.title(f"Residuals vs Fitted — {choice['spec']}")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

### Insight 2
Predicted log-frequencies remain stable across 2020–2024, confirming that the bc×dw interaction captures structural differences without temporal drift.
- Building class: Frequency decreases with class level (1→3), consistent across years; exposure concentrated in class 1.
- Dwelling type: HR shows slightly lower frequency than BR; stable pattern and larger exposure share.
- Area band: Frequency rises with area; parallel slopes across years indicate no time effect.

Residual diagnostics
Residuals show no trend or heteroscedasticity. Deviance residuals are centered near 0 with mild variance growth at higher fitted means, consistent with Poisson assumptions.

Conclusion: The Poisson GLM (area=banded | bc=Y | dw=Y | inter=bc×dw) is statistically sound and temporally stable, suitable for pricing segmentation without evidence of misspecification.

In [ ]:
# Trend and calibration sanity check 
year_trend = (
    df_hist
    .assign(pred=choice_res.mu)
    .groupby("year", observed=True)
    .agg(
        obs_claims=("claims_number", "sum"),
        exp=("exposure", "sum"),
        pred_claims=("pred", "sum")
    )
    .assign(
        obs_freq=lambda d: d["obs_claims"] / d["exp"],
        pred_freq=lambda d: d["pred_claims"] / d["exp"]
    )
)

print("Exposure-weighted frequency trend by year:")
print(year_trend[["obs_freq", "pred_freq"]].round(6))

# Plot observed vs predicted frequency trend
plt.figure(figsize=(6,4))
plt.plot(year_trend.index, year_trend["obs_freq"], marker="o", label="Observed frequency", color="black")
plt.plot(year_trend.index, year_trend["pred_freq"], marker="o", label="Predicted frequency", color="steelblue")
plt.xticks(year_trend.index.astype(int))
plt.title("Trend and calibration sanity check")
plt.xlabel("Year")
plt.ylabel("Frequency (claims / exposure)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(title="Source", framealpha=0.4)
plt.tight_layout()
plt.show()

#### Observations
Check indicates a year trend is ignored in the current model. Since time interaction with area_band was the only significantly improving coefficient (~4 ΔAIC), an area_bandXyear interaction is included in the final model.

In [ ]:
# ===== FINAL MODEL with numeric time slope =====
# final non-time spec here:
USE_BANDED_AREA = True 
INCLUDE_BC = True
INCLUDE_DW = True
INTERACTIONS = {'bcXdw', 'bcXarea'}

# Build base (non-time) design for TRAIN (2020–2024)
X_base = make_X(
    df_hist,
    use_area_band=USE_BANDED_AREA,
    include_bc=INCLUDE_BC,
    include_dw=INCLUDE_DW,
    interactions=INTERACTIONS
)

# Add numeric time slope(s): t and (optionally) area×t
# Set include_areaXt=False if you only want a plain time slope.
X_final, BASE_YEAR = add_time_trend(
    X_base, df_hist,
    include_t=True,
    include_areaXt=True,
    area_banded=USE_BANDED_AREA
)

# Fit Poisson with offset (as before)
y   = df_hist["claims_number"]
exp = df_hist["exposure"]

final_model = sm.GLM(y, X_final, family=sm.families.Poisson(), offset=np.log(exp)).fit()

print("\n=== FINAL TIME-TREND MODEL SUMMARY ===")
print(final_model.summary())
print(f"\nAnchored base_year used for t: {BASE_YEAR}")

In [ ]:
# Model valdiation (final time-calibrated model)
df_pred = df_hist.copy()
df_pred['mu_hat'] = final_model.predict(X_final, offset=np.log(df_pred['exposure']))

def summarize_factor(df, factor):
    g = df.groupby([factor, 'year'], as_index=False, observed=True).agg(
        mu_hat_sum=('mu_hat', 'sum'),
        exposure_sum=('exposure', 'sum')
    )
    g['freq_hat'] = g['mu_hat_sum'] / g['exposure_sum']
    return g

factors = ['building_class', 'dwelling_type', 'area_band']
fig, axes = plt.subplots(1, 3, figsize=(17, 3), sharey=True)

for ax, factor in zip(axes, factors):
    df_fac = summarize_factor(df_pred, factor)
    if factor == 'building_class':
        order = ['1', '2', '3']
        df_fac[factor] = df_fac[factor].astype(str)
    elif factor == 'dwelling_type':
        order = ['BR', 'HR']
    else:
        order = [str(c) for c in pd.unique(df_fac[factor])]

    df_fac[factor] = pd.Categorical(df_fac[factor], categories=order, ordered=True)

    sns.lineplot(
        data=df_fac,
        x=factor, y=np.log(df_fac['freq_hat']),
        hue='year', marker='o', ax=ax, palette='tab10',
        sort=False, estimator=None, legend=(ax is axes[0])
    )
    ax.set_xlim(-0.5, len(order) - 0.5)
    ax.set_xlabel(factor.replace('_', ' ').title())
    ax.set_ylabel('log(Expected Frequency)')
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    ax.grid(True, linestyle='--', alpha=0.4)
    ax.set_title(factor.replace('_', ' ').title())

    ax2 = ax.twinx()
    sns.barplot(data=df_fac, x=factor, y='exposure_sum',
                order=order, ax=ax2, color='grey', alpha=0.2, errorbar=None)
    ax2.set_ylabel('Exposure (sum)')
    ax2.set_yscale('log')

leg = axes[0].legend(title='Year', loc='upper left', framealpha=0.35, fontsize=8)
if leg: leg.get_frame().set_edgecolor('none')

plt.tight_layout()
plt.show()

In [ ]:
# Portfolio-level check 
obs_freq = df_hist['claims_number'].sum() / df_hist['exposure'].sum()
pred_freq = final_model.mu.sum() / df_hist['exposure'].sum()

print(f"Observed portfolio frequency: {obs_freq:.4f}")
print(f"Predicted portfolio frequency: {pred_freq:.4f}")
print(f"Deviation: {(pred_freq/obs_freq - 1)*100:.2f}%")

In [ ]:
# Segment-level checks
for var in ['building_class', 'dwelling_type', 'area_band']:
    check = (
        df_hist
        .assign(pred=final_model.mu)
        .groupby(var)
        .agg(obs_freq=('claims_number', lambda x: x.sum()),
             exp=('exposure', 'sum'),
             pred_freq=('pred', 'sum'))
        .assign(obs_rate=lambda d: d['obs_freq']/d['exp'],
                pred_rate=lambda d: d['pred_freq']/d['exp'],
                deviation=lambda d: (d['pred_rate']/d['obs_rate'] - 1)*100)
    )
    print(f"\n--- {var} ---")
    print(check[['obs_rate','pred_rate','deviation']].round(5))

In [ ]:
# Trend and calibration sanity check
trend = df_hist.copy()
trend['pred_freq'] = final_model.predict(X_final, offset=np.log(trend['exposure'])) / trend['exposure']

trend_check = (
    trend.groupby('year', as_index=False)
         .agg(obs_freq=('claims_number', lambda x: x.sum() / x.count()),
              pred_freq=('pred_freq', 'mean'))
)

plt.figure(figsize=(6,4))
plt.plot(trend_check['year'], trend_check['obs_freq'], 'o-', label='Observed frequency', color='black')
plt.plot(trend_check['year'], trend_check['pred_freq'], 'o-', label='Predicted frequency', color='steelblue')
plt.xticks(year_trend.index.astype(int))
plt.title('Trend and calibration sanity check')
plt.xlabel('Year')
plt.ylabel('Frequency (claims / exposure)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(title='Source', framealpha=0.5)
plt.show()

In [ ]:
## Save key model specs to disk (includes time-trend info)
res = final_model

frozen_model = {
    "spec": {
        "use_area_band": USE_BANDED_AREA,
        "include_bc": INCLUDE_BC,
        "include_dw": INCLUDE_DW,
        "interactions": sorted(list(INTERACTIONS)),
        "time": {
            "base_year": int(BASE_YEAR),
            "include_t": True,
            "include_areaXt": True
        }
    },
    "family": "Poisson",
    "aic": float(res.aic),
    "bic": float(getattr(res, "bic", (np.log(res.nobs) * len(res.params) - 2.0 * res.llf))),
    "pearson_over_df": float(res.pearson_chi2 / res.df_resid),
    "k_params": int(len(res.params)),
    "params": {k: float(v) for k, v in res.params.items()},
    "exog_names": list(res.model.exog_names),
}

out = "final_model_spec.json"
import json
with open(out, "w") as f:
    json.dump(frozen_model, f, indent=2)
print(f"Saved: {out}")

## Model Prediction

In [ ]:
## Predict 2025 expected claims using the frozen spec
assert (df_2025['exposure'] > 0).all(), "2025 contains non-positive exposure; cannot use log-offset."

# 2) Rebuild 2025 base X with the same mains & interactions
use_area_band = frozen_model["spec"]["use_area_band"]
include_bc    = frozen_model["spec"]["include_bc"]
include_dw    = frozen_model["spec"]["include_dw"]
interactions  = set(frozen_model["spec"]["interactions"])

X_2025_base = make_X(
    df_2025,
    use_area_band=use_area_band,
    include_bc=include_bc,
    include_dw=include_dw,
    interactions=interactions
)

# 3) Add the same time slopes with the same base_year and flags
time_cfg = frozen_model["spec"]["time"]
X_2025, _ = add_time_trend(
    X_2025_base, df_2025,
    base_year=time_cfg["base_year"],
    include_t=time_cfg["include_t"],
    include_areaXt=time_cfg["include_areaXt"],
    area_banded=use_area_band
)

# 4) Align columns and predict μ̂
exog_names = frozen_model["exog_names"]
X_2025 = X_2025.reindex(columns=exog_names, fill_value=0.0)

df_2025['mu_hat'] = final_model.predict(X_2025, offset=np.log(df_2025['exposure']))

print("Sample 2025 predictions:")
print(df_2025[['policy_id','building_class','dwelling_type','area','year','exposure','mu_hat']].head(10))

In [ ]:
## === Case Questions & Answers ===

# --- Q1: Total expected claim cost and premium at 70% LR ---
PB_2025   = 58_800   # SEK per claim
TARGET_LR = 0.70

df_2025["expected_claim_cost"] = df_2025["mu_hat"] * PB_2025
df_2025["premium_proposal"]    = df_2025["expected_claim_cost"] / TARGET_LR

total_expected_cost  = df_2025["expected_claim_cost"].sum()
total_premium_needed = total_expected_cost / TARGET_LR

print("\nQ1: Total expected claim cost and premium at 70% LR")
print(f"Total expected claim cost (2025): {total_expected_cost:,.0f} SEK")
print(f"Total premium for LR={TARGET_LR:.0%}: {total_premium_needed:,.0f} SEK")

# --- Q2: Highest-premium policy ---
cols_show = ['policy_id','building_class','dwelling_type','area','year','exposure','mu_hat','expected_claim_cost','premium_proposal']
top = df_2025.loc[df_2025['premium_proposal'].idxmax(), cols_show]
print("\nQ2: Highest-premium policy (labeled):")
for k, v in top.items():
    print(f"  {k}: {v}")

# --- Q3: Estimators confidence levels ---
print('\nQ3: Estimators confidence levels\n')
cov = final_model.cov_params()
X   = X_2025.values

# Var(η̂) = X Σ Xᵀ
var_eta = np.einsum('ij,jk,ik->i', X, cov, X)
se_eta  = np.sqrt(np.maximum(var_eta, 0.0))  # guard tiny negatives

# Delta method to μ̂: Var(μ̂) ≈ (dμ/dη)^2 Var(η) = μ̂^2 Var(η)
mu_hat = df_2025['mu_hat'].values
se_mu  = mu_hat * se_eta
cv_mu  = np.divide(se_mu, mu_hat, out=np.full_like(se_mu, np.nan), where=mu_hat>0)

df_2025['se_eta'] = se_eta
df_2025['se_mu']  = se_mu
df_2025['cv_mu']  = cv_mu   # coefficient of variation on μ̂

cols = ['policy_id','building_class','dwelling_type','area','year','exposure','mu_hat','se_eta','se_mu','cv_mu']
print(df_2025[cols].head(10))

print("\nMost certain (lowest cv_mu among μ̂>0):")
mask = df_2025['mu_hat'] > 0
print(df_2025[mask].nsmallest(10, 'cv_mu')[cols])

print("\nLeast certain (highest cv_mu among μ̂>0):")
print(df_2025[mask].nlargest(10, 'cv_mu')[cols])

# Case Q&A
1)  Based on the selected Poisson GLM model (area_banded + bc + dw main effects + bc*area + area_banded*year interactions), the expected claim cost for 2025 would be ~85.8 mSEK. Moreover, based on the Prisbasbelopp 2025 at 58'800 SEK per claim and the target LR at 70%, the minimum total premium required would be ~122.5 mSEK. These figures imply an average technical premium about 43 % above expected claims, in line with the target profitability level.These results assume the exposure and portfolio composition in 2025 are consistent with historical data. Any significant change would affect those projected estimates.

2) The highest premium corresponds to a Building Class 1, Brick (BR) dwelling of area ≈ 2 650 m².
Its predicted claim frequency is around 0.082 claims / exposure, leading to an expected claim cost ≈ 4.8 kSEK and a technical premium ≈ 6.9 kSEK at the 70 % LR. This profile lies among the largest and most exposed risks in the dataset—hence the higher expected loss and premium.

3) Uncertainty was quantified using the GLM covariance matrix:  
    Var(η̂)=X Σ Xᵀ, with the delta method giving the standard error and coefficient of variation (cv = se/μ̂).
    - Most certain predictions (cv ≈ 6 %) come from Building Class 1 – HR dwellings with medium-to-large areas, well-represented in the training data.
    - Least certain (cv ≈ 19 %) appear for small Building Class 3 – HR dwellings, a relatively rare and low-exposure segment.

In short, the model is most confident in frequent, data-rich profiles and least confident in sparse, low-area risks—a pattern typical of portfolio GLMs. High-uncertainty cells could later receive credibility adjustments or pooling in tariff refinement. For high pred_se (or relative uncertainty) → this policy’s premium is less credible, likely because it sits in a sparse region of the data (rare combination of factors). For low pred_se → prediction is stable, many similar observations support it.